## Price - Data

In order to simulate the participation in the day - ahead market as well as in the balancing market, historical pricing data is obtained from Energy Data DK. Two datastreams called "Nordpool" and "EnergiNet Balancing Prices" are taken into account to cover information on day-ahead and balancing prices on an hourly basis. The entire year of 2022 is exported and cleaned (2022-01-01 00:00:00 - 2022-12-31 23:00:00) for the so called tariff zone "DK2", where Bornholm is located. Only prices in euros are considered and negative prices are neglected accordingly to the assignment's task description. The balancing prices of "EnergiNet Balancing Prices" are available at any given time in the observed timeframe, while three full days were missing in the "Nordpool" dataset. These day-ahead prices for 06.01.2022 - 08.01.2022 were alternatively obtained from ENTSOE's webpage (link: https://transparency.entsoe.eu/dashboard/show; European Network of Transmission System Operators for Electricity). The final result of this script is a csv-file (prices_merged_df_output.csv) containing any hourly day-ahead as well as balancing prices (Up and Down Balancing) in Euros for the year 2022.

 https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=30.10.2022+00:00|CET|DAY&biddingZone.values=CTY|10Y1001A1001A65H!BZN|10YDK-2--------M&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)



In [7]:
import pandas as pd
import numpy as np


# Class DataManagement covers the reading of the raw data in the constructor and provides two seperate datahandling-functions for the two seperate data sets.
class DataManagement():
    
    
    # The constructor reads the associated raw files and calls the corresponding datahandling-functions.
    # if Area = DK2 the EnergiNet (balancing prices) file is handled, while if Area = Nordpool the Nordpool (day-ahead prices) file is adjusted. 
    # "dropColumns" is the list of columns, which will be dropped and not further used.
    def __init__(self, filename, Area, dropColumns):
        self.filename = filename
        if(Area=='DK2'):
            #GitHub-Path
            self.df = pd.read_csv("../Data assignment 1/"+str(filename)+".csv", delimiter=',')
            #self.df = pd.read_csv("./data/raw/"+str(filename)+".csv", delimiter=',')
        else:
            #GitHub-Path
            self.df = pd.read_csv("../Data assignment 1/"+str(filename)+".csv", delimiter=';')
            #self.df = pd.read_csv("./data/raw/"+str(filename)+".csv", delimiter=';')
        
        for column_name in self.df.columns:
                self.df[column_name] = self.df[column_name].str.replace(',', '.')
        
        
        #EnergiNet's raw dataset also covers the tariff zone DK1, which is why filtering for tariff zone DK2 is necessary
        if(Area == 'DK2'):
            self.df = self.df[self.df["PriceArea | PriceArea | 804696"]==Area]
            self.cleanDataSetEnergiNet(dropColumns)
        elif(Area == 'Nordpool'):
            self.cleanNordpool(dropColumns)
        
        
        
    # This function drops, renames and alters columns for the EnergiNet dataset. The prices are transfered from €/MWh to €/kWh.
    # After that only three columns remain: 
    # HourDK: Timestamp per hour in yyyy-mm-dd hh:mm:ss
    # BalancingPriceDownEUR: Euro price for down-balancing in €/MWh
    # BalancingPriceUpEUR: Euro price for up-balancing in €/MWh
    # Negative prices are neglected. 
    # The result is a dataframe self.df
    def cleanDataSetEnergiNet(self, dropColumns):
        self.df.rename(columns={'HourDK | HourDK | 804695':'HourDK'}, inplace=True)
        self.df['HourDK'] = pd.to_datetime(self.df['HourDK'])
        self.df = self.df[(self.df['HourDK'] >='2022-01-01 00:00:00') & ('2023-01-01 00:00:00' > self.df['HourDK'])]
        self.df.drop(dropColumns,axis=1,inplace=True)
        self.df.reset_index(drop=True, inplace=True)
        self.df.index = self.df.index + 1
        
        self.df.columns = self.df.columns.str.strip()
        self.df.rename(columns={'BalancingPowerPriceUpEUR | BalancingPowerPriceUpEUR | 804718':'BalancingPriceUpEUR', 'BalancingPowerPriceDownEUR | BalancingPowerPriceDownEUR | 804720':'BalancingPriceDownEUR'}, inplace=True)
        
        self.df['BalancingPriceUpEUR'] = pd.to_numeric(self.df['BalancingPriceUpEUR'])
        self.df['BalancingPriceUpEUR'] = (self.df['BalancingPriceUpEUR']/1000)
        self.df['BalancingPriceDownEUR'] = pd.to_numeric(self.df['BalancingPriceDownEUR'])
        self.df['BalancingPriceDownEUR'] = (self.df['BalancingPriceDownEUR']/1000)
        self.df.loc[self.df['BalancingPriceUpEUR'] < 0, 'BalancingPriceUpEUR'] = 0
        self.df.loc[self.df['BalancingPriceDownEUR'] < 0, 'BalancingPriceDownEUR'] = 0
        
        
        #self.df.to_csv("./data/processed/BalancingPrices.csv")
    
    # This function drops, renames and alters columns for the Nordpool dataset. The prices are transfered from €/MWh to €/kWh.
    # After that only two columns remain: 
    # HourDK: Timestamp per hour in yyyy-mm-dd hh:mm:ss
    # DA_PriceEUR: Euro day-ahead prices in €/MWh
    # Negative prices are neglected.
    # The result is a dataframe self.df.
    def cleanNordpool(self, dropColumns):
        self.df.drop(dropColumns,axis=1,inplace=True)
        self.df.rename(columns={'ts':'HourDK'}, inplace=True)
        self.df['HourDK'] = pd.to_datetime(self.df['HourDK'])
        self.df.rename(columns={'Nordpool Elspot Prices - hourly price DK-DK2 EUR/MWh | 9F7J/00/00/Nordpool/DK2/hourly_spot_eur | 3038':'DA_PriceEUR'}, inplace=True)
        self.df['DA_PriceEUR'] = pd.to_numeric(self.df['DA_PriceEUR'])
        self.df['DA_PriceEUR'] = (self.df['DA_PriceEUR']/1000)
        self.df.loc[self.df['DA_PriceEUR'] < 0, 'DA_PriceEUR'] = 0
        
        full_range = pd.date_range(start= self.df['HourDK'].min(), end=self.df['HourDK'].max(), freq='h')
        
        missing_timestamps = full_range.difference(self.df['HourDK'])
        
        
        #self.df.to_csv("./data/processed/DA_Prices.csv")
        
    
    
        
        

In [8]:
# dropCol shows all Columns that are not further used of the EnergiNet dataset. 
dropCol = ['ts', 
           'HourUTC | HourUTC | 804694', 
           'ImbalancePriceDKK | ImbalancePriceDKK | 804717',
           'BalancingPowerPriceUpDKK | BalancingPowerPriceUpDKK | 804719',
           'BalancingPowerPriceDownDKK | BalancingPowerPriceDownDKK | 804721',
           'mFRRDownActBal | mFRRDownActBal | 804724',
           'PriceArea | PriceArea | 804696',
           'mFRRUpActSpec | mFRRUpActSpec | 804713',
           'mFRRDownActSpec | mFRRDownActSpec | 804714',
           'mFRRUpActBal | mFRRUpActBal | 804722',
           'mFRRDownActBal | mFRRDownActBal | 804724',
           'ImbalanceMWh | ImbalanceMWh | 804715',
           'ImbalancePriceEUR | ImbalancePriceEUR | 804716'
           ]
EnergiNet = DataManagement("EnergiNet_2022","DK2",dropCol)

# dropCol shows all Columns that are not further used of the Nordpool dataset. 
dropCol = np.array([
    'Nordpool Elspot Prices - daily average price DK-DK2 DKK/MWh | 9F7J/00/00/Nordpool/DK2/daily_average_spot_dkk | 4821',
   'Nordpool Elspot Prices - daily average price DK-DK2 EUR/MWh | 9F7J/00/00/Nordpool/DK2/daily_average_spot_eur | 4822',
    'Nordpool Elspot Prices - hourly price DK-DK2 DKK/MWh | 9F7J/00/00/Nordpool/DK2/hourly_spot_dkk | 4820'
])
Nordpool = DataManagement("Nordpool_2022","Nordpool",dropCol)


# 
prices_merge_df = pd.merge(EnergiNet.df, Nordpool.df, how='outer', on="HourDK")

prices_merge_df = prices_merge_df[prices_merge_df['HourDK']>=pd.to_datetime('2022-01-01 00:00:00')]

#GitHub-Path
prices_merge_df.to_csv('../Data assignment 1/prices_merged_df_output.csv', index=False)
#prices_merge_df.to_csv('./data/processed/prices_merged_df_output.csv', index=False)


